Chapter 23 Sorting Columns and indexes


In [3]:
import pandas as pd


In [5]:
def tweak_siena_pres(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols}))


    return (df
     .rename(columns={'Seq.':'Seq'})    # 1
     .rename(columns={k:v.replace(' ', '_') for k,v in
        {'Bg': 'Background',
         'PL': 'Party leadership', 'CAb': 'Communication ability',
         'RC': 'Relations with Congress', 'CAp': 'Court appointments',
         'HE': 'Handling of economy', 'L': 'Luck',
         'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
         'EAp': 'Executive appointments', 'OA': 'Overall ability',
         'Im': 'Imagination', 'DA': 'Domestic accomplishments',
         'Int': 'Integrity', 'EAb': 'Executive ability',
         'FPA': 'Foreign policy accomplishments',
         'LA': 'Leadership ability',
         'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
         'EV': "Experts' view", 'O': 'Overall'}.items()})
     .astype({'Party':'category'})  # 2
     .pipe(int64_to_uint8)  # 3
     .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8') # 4
                 .sum(axis=1).rank(method='dense').astype('uint8')),
             Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                 labels='1st 2nd 3rd 4th'.split())
            )
    )

In [3]:
# version 2 , simplified the columns assignment, combined and modified reaname(), move rename() with underscore
def tweak_siena_pres_2(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols}))

    return (df
     .rename(columns=
        {'Seq.':'Seq',
         'Bg': 'Background',
         'PL': 'Party leadership', 'CAb': 'Communication ability',
         'RC': 'Relations with Congress', 'CAp': 'Court appointments',
         'HE': 'Handling of economy', 'L': 'Luck',
         'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
         'EAp': 'Executive appointments', 'OA': 'Overall ability',
         'Im': 'Imagination', 'DA': 'Domestic accomplishments',
         'Int': 'Integrity', 'EAb': 'Executive ability',
         'FPA': 'Foreign policy accomplishments',
         'LA': 'Leadership ability',
         'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
         'EV': "Experts' view", 'O': 'Overall'})
     .rename(columns = lambda c:c.replace(' ','_'))
     .astype({'Party':'category'})  # 2
     .pipe(int64_to_uint8)  # 3
     .assign(Average_rank=lambda df_:df_.select_dtypes('uint8') # 4
                 .sum(axis=1).rank(method='dense').astype('uint8'),
             Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                 labels='1st 2nd 3rd 4th'.split())
            )
    )

In [6]:
pd.set_option('display.max_rows', 10)             
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    'siena2018-pres.csv'
df = pd.read_csv(url, index_col=0)

In [7]:
pres = tweak_siena_pres(df)

In [8]:
pres.sort_values(by='Party')


,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
22,22/24,Grover Cleveland,Democratic,26,23,26,27,19,27,22,...,20,21,23,23,21,15,22,23,23,3rd
31,32,Franklin D. Roosevelt,Democratic,6,3,16,12,5,3,4,...,2,2,3,3,1,4,3,2,2,1st
17,17,Andrew Johnson,Democratic,42,42,41,42,40,34,43,...,44,43,42,43,41,43,43,44,44,4th
32,33,Harry S. Truman,Democratic,31,16,9,21,12,8,12,...,17,8,10,7,4,9,7,9,9,1st
15,15,James Buchanan,Democratic,36,43,40,39,42,41,40,...,43,42,43,44,43,44,44,43,43,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,18,Ulysses S. Grant,Republican,20,24,25,24,26,18,17,...,24,26,38,24,24,31,24,24,24,3rd
44,45,Donald Trump,Republican,43,40,44,44,10,25,42,...,40,39,44,40,42,41,42,42,42,4th
13,13,Millard Fillmore,Whig,40,37,36,38,35,38,32,...,39,37,37,37,37,33,37,38,39,4th
12,12,Zachary Taylor,Whig,30,26,22,32,37,24,26,...,37,27,33,27,30,26,30,30,30,3rd


In [9]:
(pres
 .sort_values(by=['Party','Average_rank'],
              ascending=[True, False]))

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
17,17,Andrew Johnson,Democratic,42,42,41,42,40,34,43,...,44,43,42,43,41,43,43,44,44,4th
15,15,James Buchanan,Democratic,36,43,40,39,42,41,40,...,43,42,43,44,43,44,44,43,43,4th
14,14,Franklin Pierce,Democratic,38,39,38,40,39,38,39,...,41,40,39,41,39,38,40,40,41,4th
38,39,Jimmy Carter,Democratic,25,19,3,15,38,27,29,...,19,38,22,28,25,34,26,26,27,3rd
8,8,Martin Van Buren,Democratic,23,22,27,25,34,28,20,...,25,31,26,29,27,24,28,25,25,3rd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26,Theodore Roosevelt,Republican,5,4,8,6,2,2,15,...,9,3,5,4,3,5,4,4,4,1st
16,16,Abraham Lincoln,Republican,28,1,2,2,18,1,1,...,3,4,2,1,6,2,1,3,3,1st
13,13,Millard Fillmore,Whig,40,37,36,38,35,38,32,...,39,37,37,37,37,33,37,38,39,4th
9,9,William Henry Harrison,Whig,22,38,28,37,44,32,41,...,42,41,40,42,44,37,39,39,38,4th


In [10]:
(pres
 .President
 .str.split()
)

1       [George, Washington]
2              [John, Adams]
3        [Thomas, Jefferson]
4           [James, Madison]
5            [James, Monroe]
               ...          
40    [George, H., W., Bush]
41           [Bill, Clinton]
42        [George, W., Bush]
43           [Barack, Obama]
44           [Donald, Trump]
Name: President, Length: 44, dtype: object

In [12]:
(pres
 .President
 .str.split()
 .apply(lambda val: val[-1]))

1     Washington
2          Adams
3      Jefferson
4        Madison
5         Monroe
         ...    
40          Bush
41       Clinton
42          Bush
43         Obama
44         Trump
Name: President, Length: 44, dtype: object

In [13]:
(pres
 .sort_values(by='President',
              key=lambda nam_ser: nam_ser
                    .str.split()
                    .apply(lambda val:val[-1])))

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
2,2,John Adams,Federalist,3,13,4,4,24,14,31,...,4,13,15,19,13,16,10,14,13,2nd
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,15,17,18,21,15,14,18,18,18,2nd
21,21,Chester A. Arthur,Republican,41,31,37,36,17,33,22,...,33,30,31,25,32,23,31,34,34,4th
15,15,James Buchanan,Democratic,36,43,40,39,42,41,40,...,43,42,43,44,43,44,44,43,43,4th
8,8,Martin Van Buren,Democratic,23,22,27,25,34,28,20,...,25,31,26,29,27,24,28,25,25,3rd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,33,Harry S. Truman,Democratic,31,16,9,21,12,8,12,...,17,8,10,7,4,9,7,9,9,1st
44,45,Donald Trump,Republican,43,40,44,44,10,25,42,...,40,39,44,40,42,41,42,42,42,4th
10,10,John Tyler,Independent,34,33,35,34,22,26,37,...,38,34,36,36,26,32,36,37,37,4th
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st


In [14]:
(pres.
 set_index('President')
 .sort_index())

,Seq,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
President,,,,,,,,,,,,,,,,,,,,,
Abraham Lincoln,16,Republican,28,1,2,2,18,1,1,1,...,3,4,2,1,6,2,1,3,3,1st
Andrew Jackson,7,Democratic,37,15,29,28,4,4,38,11,...,30,25,25,17,23,20,19,19,19,2nd
Andrew Johnson,17,Democratic,42,42,41,42,40,34,43,43,...,44,43,42,43,41,43,43,44,44,4th
Barack Obama,44,Democratic,24,11,13,9,15,23,16,18,...,14,10,13,13,20,10,11,17,17,2nd
Benjamin Harrison,23,Republican,33,34,30,35,28,36,33,35,...,35,32,34,32,29,29,33,35,36,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
William Henry Harrison,9,Whig,22,38,28,37,44,32,41,38,...,42,41,40,42,44,37,39,39,38,4th
William Howard Taft,27,Republican,12,28,12,14,27,31,19,23,...,16,19,21,18,22,19,23,22,22,2nd
William McKinley,25,Republican,29,20,20,26,32,22,21,17,...,23,16,17,20,14,13,20,20,20,2nd


In [15]:
# this will display a error, because of duplicated index values that 
# must be sorted before slicing

(pres
 .set_index('Party')
 .loc['Democratic':'Republican'])

KeyError: "Cannot get left slice bound for non-unique label: 'Democratic'"

In [16]:
(pres
 .set_index('Party')
 .sort_index()
 .loc['Democratic':'Republican']
)

,Seq,President,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,...,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile
Party,,,,,,,,,,,,,,,,,,,,,
Democratic,22/24,Grover Cleveland,26,23,26,27,19,27,22,19,...,20,21,23,23,21,15,22,23,23,3rd
Democratic,32,Franklin D. Roosevelt,6,3,16,12,5,3,4,3,...,2,2,3,3,1,4,3,2,2,1st
Democratic,17,Andrew Johnson,42,42,41,42,40,34,43,43,...,44,43,42,43,41,43,43,44,44,4th
Democratic,33,Harry S. Truman,31,16,9,21,12,8,12,8,...,17,8,10,7,4,9,7,9,9,1st
Democratic,15,James Buchanan,36,43,40,39,42,41,40,42,...,43,42,43,44,43,44,44,43,43,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Republican,26,Theodore Roosevelt,5,4,8,6,2,2,15,4,...,9,3,5,4,3,5,4,4,4,1st
Republican,25,William McKinley,29,20,20,26,32,22,21,17,...,23,16,17,20,14,13,20,20,20,2nd
Republican,23,Benjamin Harrison,33,34,30,35,28,36,33,35,...,35,32,34,32,29,29,33,35,36,4th


In [ ]:
#23.5 Exercises

